Financial Risk Prediction Using Machine Learning

In [ ]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Number of samples
n = 500

# Generate synthetic financial data
data = {
    'loan_amount': np.random.randint(1000, 50000, n),
    'annual_income': np.random.randint(20000, 200000, n),
    'credit_score': np.random.randint(300, 850, n),
    'employment_years': np.random.randint(0, 30, n),
    'home_ownership': np.random.choice(['OWN', 'MORTGAGE', 'RENT'], n),
    'loan_purpose': np.random.choice(['debt_consolidation', 'credit_card', 'home_improvement', 'car', 'education'], n),
    'dti_ratio': np.random.uniform(0, 40, n),  # debt-to-income ratio
    'num_of_open_accounts': np.random.randint(1, 20, n),
    'delinq_2yrs': np.random.randint(0, 5, n),
    'revol_util': np.random.uniform(0, 100, n),  # credit utilization
}

# Add target column: 'default' (1 = default, 0 = not)
# Higher loan amount + lower credit score + high DTI increases default risk
prob_default = (
    0.3 * (data['loan_amount'] / 50000) +
    0.4 * (1 - (np.array(data['credit_score']) / 850)) +
    0.3 * (np.array(data['dti_ratio']) / 40)
)
data['default'] = np.random.binomial(1, prob_default)

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('financial_risk_data.csv', index=False)

print("✅ 'financial_risk_data.csv' created successfully with 500 rows!")
print(df.head())


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Step 1: Load dataset
df = pd.read_csv("financial_risk_data.csv")

print("🔹 Dataset Loaded Successfully!")
print("Shape:", df.shape)
print(df.head())

# Step 2: Encode categorical variables
categorical_cols = ['home_ownership', 'loan_purpose']
encoder = LabelEncoder()

for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])

print("\n✅ Encoded categorical columns:", categorical_cols)

# Step 3: Split features (X) and target (y)
X = df.drop("default", axis=1)
y = df["default"]

# Step 4: Scale numeric features
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

print("\n✅ Scaled numeric columns!")

# Step 5: Split train/test data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("\n✅ Data Split Completed!")
print(f"🔹 X_train: {X_train.shape}")
print(f"🔹 X_test: {X_test.shape}")
print(f"🔹 y_train: {y_train.shape}")
print(f"🔹 y_test: {y_test.shape}")

# Step 6: Save feature names for later use (important for prediction)
feature_names = X_train.columns
print("\n📋 Saved feature names for future prediction!")
print(feature_names.tolist())


🔹 Dataset Loaded Successfully!
Shape: (500, 11)
   loan_amount  annual_income  credit_score  employment_years home_ownership  \
0        16795         124185           599                22       MORTGAGE   
1         1860          43049           625                 9            OWN   
2        39158         118116           709                 9            OWN   
3        45732         117379           417                10           RENT   
4        12284          88385           524                20           RENT   

         loan_purpose  dti_ratio  num_of_open_accounts  delinq_2yrs  \
0                 car   5.678880                    13            0   
1  debt_consolidation  13.917656                     4            4   
2    home_improvement  18.014039                     2            3   
3  debt_consolidation  29.953053                    16            0   
4                 car  26.045903                     9            0   

   revol_util  default  
0   42.333514      

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Step 1: Train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Step 2: Evaluate the model
y_pred = model.predict(X_test)

print("\n✅ Model training complete!")
print("\n🎯 Model Accuracy:", accuracy_score(y_test, y_pred) * 100, "%")

print("\n📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\n📋 Classification Report:\n", classification_report(y_test, y_pred))

# Step 3: Safe Prediction (with correct feature order)
import pandas as pd

# Example: new loan applicant data (you can change values)
sample_data = {
    'loan_amount': [25000],
    'annual_income': [85000],
    'credit_score': [680],
    'employment_years': [5],
    'home_ownership': [1],       # 0=RENT, 1=OWN, 2=MORTGAGE (depends on encoding)
    'loan_purpose': [2],         # 0=car, 1=debt_consolidation, 2=home_improvement etc.
    'dti_ratio': [18.5],
    'num_of_open_accounts': [8],
    'delinq_2yrs': [1],
    'revol_util': [45.0]
}

# Convert to DataFrame
sample_df = pd.DataFrame(sample_data)

# ⚙️ Match feature names and scale numeric columns
sample_df = sample_df.reindex(columns=feature_names, fill_value=0)
sample_df[numeric_cols] = scaler.transform(sample_df[numeric_cols])

# Step 4: Predict risk level
pred = model.predict(sample_df)[0]
risk_label = "⚠️ Default Risk" if pred == 1 else "✅ Low Risk"

print("\n🔮 Predicted Risk Level:", risk_label)



✅ Model training complete!

🎯 Model Accuracy: 56.00000000000001 %

📊 Confusion Matrix:
 [[43 20]
 [24 13]]

📋 Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.68      0.66        63
           1       0.39      0.35      0.37        37

    accuracy                           0.56       100
   macro avg       0.52      0.52      0.52       100
weighted avg       0.55      0.56      0.55       100


🔮 Predicted Risk Level: ✅ Low Risk
